In [226]:
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [166]:
train = pd.read_csv('/home/aleksandr/data/kaggle/project/train.csv')
clien_profile = pd.read_csv('/home/aleksandr/data/kaggle/project/client_profile.csv')
bki = pd.read_csv('/home/aleksandr/data/kaggle/project/bki.csv')
app_hist = pd.read_csv('/home/aleksandr/data/kaggle/project/applications_history.csv')
payments = pd.read_csv('/home/aleksandr/data/kaggle/project/payments.csv')
# sample_submit = pd.read_csv('/home/aleksandr/data/kaggle/project/sample_submit.csv')
# test = pd.read_csv('/home/aleksandr/data/kaggle/project/test.csv')

In [3]:
def info_df(i_df: DataFrame):
    print('Размер: ', i_df.shape, '\n')
    print(i_df.head(), '\n')
    print('Заголовки', i_df.columns, '\n')
    print(i_df.describe())

In [54]:
info_df(train)

Размер:  (110093, 3) 

   APPLICATION_NUMBER  TARGET NAME_CONTRACT_TYPE
0           123687442       0               Cash
1           123597908       1               Cash
2           123526683       0               Cash
3           123710391       1               Cash
4           123590329       1               Cash 

Заголовки Index(['APPLICATION_NUMBER', 'TARGET', 'NAME_CONTRACT_TYPE'], dtype='object') 

       APPLICATION_NUMBER         TARGET
count        1.100930e+05  110093.000000
mean         1.235772e+08       0.080813
std          8.880939e+04       0.272550
min          1.234233e+08       0.000000
25%          1.235005e+08       0.000000
50%          1.235769e+08       0.000000
75%          1.236539e+08       0.000000
max          1.237308e+08       1.000000


Файл "train" содержит данные по заявкам на кредит:
    - APPLICATION_NUMBER - номер заявки;
    - TARGET - одобрена или нет заявка (0 - не одобрена, 1 одобрена);
    - NAME_CONTRACT_TYPE - тип контракта (наличные или кредитная карта).

In [55]:
info_df(clien_profile)

Размер:  (250000, 24) 

   APPLICATION_NUMBER GENDER  CHILDRENS  TOTAL_SALARY  AMOUNT_CREDIT  \
0           123666076      F          0      157500.0       270000.0   
1           123423688      F          0      270000.0       536917.5   
2           123501780      M          1      427500.0       239850.0   
3           123588799      M          0      112500.0       254700.0   
4           123647485      M          0      130500.0       614574.0   

   AMOUNT_ANNUITY                EDUCATION_LEVEL   FAMILY_STATUS  \
0         13500.0              Incomplete higher  Civil marriage   
1         28467.0  Secondary / secondary special         Married   
2         23850.0              Incomplete higher         Married   
3         17149.5  Secondary / secondary special         Married   
4         19822.5                Lower secondary         Married   

   REGION_POPULATION    AGE  ...  FAMILY_SIZE  EXTERNAL_SCORING_RATING_1  \
0           0.008068   8560  ...          2.0             

In [56]:
info_df(app_hist)

Размер:  (1670214, 26) 

   PREV_APPLICATION_NUMBER  APPLICATION_NUMBER NAME_CONTRACT_TYPE  \
0                 49298709           123595216                NaN   
1                 50070639           123431468               Cash   
2                 49791680           123445379               Cash   
3                 50087457           123499497               Cash   
4                 49052479           123525393               Cash   

   AMOUNT_ANNUITY  AMT_APPLICATION  AMOUNT_CREDIT  AMOUNT_PAYMENT  \
0        1730.430          17145.0        17145.0             0.0   
1       25188.615         607500.0       679671.0             NaN   
2       15060.735         112500.0       136444.5             NaN   
3       47041.335         450000.0       470790.0             NaN   
4       31924.395         337500.0       404055.0             NaN   

   AMOUNT_GOODS_PAYMENT NAME_CONTRACT_STATUS  DAYS_DECISION  ...  \
0               17145.0             Approved             73  ...   
1        

In [57]:
info_df(bki)

Размер:  (945234, 17) 

   APPLICATION_NUMBER  BUREAU_ID CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0           123538884    5223613        Active      currency 1        718.0   
1           123436670    6207544        Closed      currency 1        696.0   
2           123589020    6326395        Closed      currency 1        165.0   
3           123494590    6606618        Active      currency 1         55.0   
4           123446603    5046832        Active      currency 1        358.0   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0                377.0                NaN   
1                   0                511.0              511.0   
2                   0                149.0              160.0   
3                   0                310.0                NaN   
4                   0                 35.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                19386.81                   0

In [58]:
info_df(payments)

Размер:  (1023932, 8) 

   PREV_APPLICATION_NUMBER  APPLICATION_NUMBER  NUM_INSTALMENT_VERSION  \
0                 49011181           123664960                     1.0   
1                 48683432           123497205                     1.0   
2                 48652024           123749925                     1.0   
3                 48398897           123550846                     0.0   
4                 49867197           123562174                     0.0   

   NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  \
0                      5           1002.0              1015.0       12156.615   
1                     13            442.0               432.0       18392.535   
2                     10              8.0                23.0        5499.945   
3                     82            398.0               398.0        7082.145   
4                     63           1359.0              1359.0         156.735   

   AMT_PAYMENT  
0    12156.615  
1    10047

In [28]:
print(' train:     ', len(train['APPLICATION_NUMBER'].unique()), train.shape[0], '\n',
     'clien_profile: ', len(clien_profile['APPLICATION_NUMBER'].unique()), clien_profile.shape[0], '\n',
     'app_hist:      ', len(app_hist['APPLICATION_NUMBER'].unique()), app_hist.shape[0], '\n',
     'bki:           ', len(bki['APPLICATION_NUMBER'].unique()), bki.shape[0], '\n',
     'payments:      ', len(payments['APPLICATION_NUMBER'].unique()), payments.shape[0], '\n',
     )

 train:      110093 110093 
 clien_profile:  250000 250000 
 app_hist:       338857 1670214 
 bki:            273131 945234 
 payments:       264726 1023932 



In [13]:
train['APPLICATION_NUMBER'].value_counts()

123716209    1
123441115    1
123709406    1
123711455    1
123619298    1
            ..
123669812    1
123667765    1
123671863    1
123432248    1
123469824    1
Name: APPLICATION_NUMBER, Length: 110093, dtype: int64

Объединим датасет train и профили клиентов по номеру заявки

In [167]:
train.shape

(110093, 3)

In [168]:
train = train.join(clien_profile.set_index('APPLICATION_NUMBER'), on='APPLICATION_NUMBER', how='left')

In [169]:
train.shape

(110093, 26)

In [170]:
train.head()

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,0,Cash,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,...,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0
1,123597908,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123526683,0,Cash,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,...,2.0,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0
3,123710391,1,Cash,M,0.0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,...,2.0,NaN,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0
4,123590329,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
train.describe(include='all')

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,1.100930e+05,110093.000000,110093,89539,89539.000000,8.953900e+04,8.953900e+04,89534.000000,89539,89539,...,89538.000000,39091.000000,89345.000000,71808.000000,77480.000000,77480.000000,77480.000000,77480.000000,77480.000000,77480.000000
unique,NaN,NaN,2,3,NaN,NaN,NaN,NaN,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Cash,F,NaN,NaN,NaN,NaN,Secondary / secondary special,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,99551,59077,NaN,NaN,NaN,NaN,63506,57236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.235772e+08,0.080813,NaN,NaN,0.421034,1.683230e+05,5.997495e+05,27119.655449,NaN,NaN,...,2.157844,0.501400,0.513890,0.511020,0.006066,0.006776,0.034151,0.266585,0.265333,1.903601
std,8.880939e+04,0.272550,NaN,NaN,0.728803,9.962278e+04,4.031694e+05,14503.018565,NaN,NaN,...,0.915425,0.209929,0.191253,0.195153,0.083264,0.111512,0.206492,0.905296,0.614651,1.875062
min,1.234233e+08,0.000000,NaN,NaN,0.000000,2.565000e+04,4.500000e+04,1615.500000,NaN,NaN,...,1.000000,0.014691,0.000010,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235005e+08,0.000000,NaN,NaN,0.000000,1.125000e+05,2.700000e+05,16524.000000,NaN,NaN,...,2.000000,0.334582,0.391285,0.370650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.235769e+08,0.000000,NaN,NaN,0.000000,1.485000e+05,5.177880e+05,24903.000000,NaN,NaN,...,2.000000,0.504836,0.565177,0.535276,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.236539e+08,0.000000,NaN,NaN,1.000000,2.025000e+05,8.086500e+05,34596.000000,NaN,NaN,...,3.000000,0.671532,0.663661,0.669057,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000


Видим, что не по всем заявкам присутствуют данные о клиентах.

Нам это поможет?

Есть ли одобренные заявки без данных о клиенте?

In [92]:
target_0 = train.loc[train['TARGET'] == 0]
target_1 = train.loc[train['TARGET'] == 1]

In [104]:
target_0.loc[target_0['GENDER'].isnull()].shape[0]/target_0.shape[0]

0.1864204118739871

In [105]:
target_1.loc[target_1['GENDER'].isnull()].shape[0]/target_1.shape[0]

0.18983927166460604

Отношение заявок с незаполненными данными к общему числу заявок в одобренных и в не одобренных примерно одинаковое, это нам не поможет в обучении модели.

In [251]:
join_index = ['APPLICATION_NUMBER']
bki.loc[bki[join_index].duplicated()]

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
597,123469853,6544366,Active,currency 1,384.0,0,347.0,NaN,NaN,0,515970.0,238090.5,0.0,0.0,Consumer credit,16.0,22563.00
928,123627360,5082283,Closed,currency 1,1515.0,0,419.0,1185.0,NaN,0,276277.5,0.0,0.0,0.0,Consumer credit,1183.0,NaN
930,123695993,5462021,Closed,currency 1,941.0,0,580.0,580.0,NaN,0,77265.0,0.0,0.0,0.0,Consumer credit,575.0,NaN
1188,123556974,6176314,Closed,currency 1,1066.0,0,761.0,765.0,NaN,0,59805.0,0.0,0.0,0.0,Consumer credit,765.0,5141.16
1818,123519026,5784895,Closed,currency 1,2749.0,0,1358.0,1743.0,18413.415,0,1104804.0,0.0,0.0,0.0,Car loan,1356.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945229,123673441,5235365,Closed,currency 1,2759.0,0,1298.0,1834.0,NaN,0,332725.5,0.0,NaN,0.0,Consumer credit,1707.0,NaN
945230,123539211,5899696,Active,currency 1,359.0,0,1467.0,NaN,NaN,0,1471500.0,1320183.0,0.0,0.0,Consumer credit,47.0,NaN
945231,123686333,5445504,Closed,currency 1,1102.0,0,725.0,370.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,233.0,NaN
945232,123508200,6679628,Active,currency 1,1579.0,0,2085.0,NaN,2339.955,0,108000.0,0.0,0.0,0.0,Credit card,16.0,NaN


Просто объединить наборы данных по номеру заявки не получится, т.к. присутствуют дубликаты. Посмотрим дубликаты для того чтобы понять как их использовать.

In [207]:
pd.set_option('display.max_rows', 103) 

In [253]:
dubl_app = bki.loc[bki[join_index].duplicated(), 'APPLICATION_NUMBER'].values
bki.loc[bki['APPLICATION_NUMBER'].isin(dubl_app)].sort_values(join_index)

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
281301,123423340,5921341,Active,currency 1,559.0,0,902.0,NaN,NaN,0,337680.000,113166.00,0.000,0.0,Consumer credit,6.0,4630.5
243230,123423340,5977045,Active,currency 1,320.0,0,411.0,NaN,NaN,0,168345.000,110281.50,NaN,0.0,Consumer credit,10.0,9364.5
540271,123423341,6164664,Closed,currency 1,645.0,0,85.0,36.0,5043.645,0,120735.000,0.00,0.000,0.0,Consumer credit,34.0,0.0
70768,123423341,6206134,Active,currency 1,103.0,0,NaN,NaN,40.500,0,31988.565,0.00,31988.565,0.0,Credit card,24.0,0.0
492691,123423341,6226978,Active,currency 1,1042.0,0,780.0,NaN,NaN,0,450000.000,245781.00,0.000,0.0,Consumer credit,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217996,123779594,5202182,Active,currency 1,363.0,0,1463.0,NaN,0.000,0,436032.000,363573.54,0.000,0.0,Consumer credit,25.0,0.0
173541,123779594,5225333,Closed,currency 1,1022.0,0,959.0,958.0,NaN,0,22995.000,0.00,0.000,0.0,Consumer credit,890.0,0.0
660891,123779594,5209371,Closed,currency 1,970.0,0,621.0,712.0,NaN,0,27472.500,0.00,0.000,0.0,Consumer credit,700.0,NaN
33351,123779594,5128415,Closed,currency 1,2337.0,0,1243.0,1621.0,16618.500,0,450000.000,NaN,NaN,0.0,Consumer credit,1621.0,0.0


In [255]:
# Есть активные кредиты
train['exists_activ_credit'] = train['APPLICATION_NUMBER'].isin(
                                                            bki.loc[bki['CREDIT_ACTIVE'] == 'Active', 
                                                                    'APPLICATION_NUMBER'].values
                                                                )

# Есть закрытые кредиты
train['exists_closed_credit'] = train['APPLICATION_NUMBER'].isin(
                                                            bki.loc[bki['CREDIT_ACTIVE'] == 'Closed', 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Были или есть кредиты в валюте 1
train['currency_1'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CREDIT_CURRENCY'] == 'currency 1', 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Были или есть кредиты в валюте 2
train['currency_2'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CREDIT_CURRENCY'] == 'currency 2', 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Были или есть кредиты в валюте 3
train['currency_3'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CREDIT_CURRENCY'] == 'currency 3', 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Были или есть кредиты в валюте 4
train['currency_4'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CREDIT_CURRENCY'] == 'currency 4', 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Были просрочки по кредиту
train['EXISTS_OVERDUE'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CREDIT_DAY_OVERDUE'] > 0, 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Былf пролонгация кредита
train['EXISTS_CREDIT_PROLONG'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CNT_CREDIT_PROLONG'] > 0, 
                                                                    'APPLICATION_NUMBER'].values
                                                                )
# Былf пролонгация кредита
train['EXISTS_CREDIT_PROLONG'] =  train['APPLICATION_NUMBER'].isin(
                                                        bki.loc[bki['CNT_CREDIT_PROLONG'] > 0, 
                                                                    'APPLICATION_NUMBER'].values
                                                                )CREDIT_TYPE

0         False
1          True
2          True
3         False
4          True
          ...  
110088    False
110089     True
110090    False
110091    False
110092     True
Name: APPLICATION_NUMBER, Length: 110093, dtype: bool

In [302]:
def add_min_max_mean_param(i_df_main, i_df, column):
    # Средний срок кредита
#     df_tmp = i_df.groupby('APPLICATION_NUMBER')[column].mean()
    i_df_main[f'{column}_mean'] = i_df.groupby('APPLICATION_NUMBER')[column].mean()
    i_df_main[f'{column}_min'] = i_df.groupby('APPLICATION_NUMBER')[column].min()
    i_df_main[f'{column}_max'] = i_df.groupby('APPLICATION_NUMBER')[column].max()
#     print(type(df_tmp))
#     df_tmp.rename(columns={ column: f'{column}_mean' }, inplace=True) 
#     re = i_df_main.join(df_tmp, on='APPLICATION_NUMBER', how='left')
#     # Минимальный срок кредита
#     df_tmp = i_df.groupby('APPLICATION_NUMBER')[column].min()
#     df_tmp.columns = f'{column}_min'
#     i_df_main.join(df_tmp, on='APPLICATION_NUMBER', how='left')
#     # Максимальный срок кредита
#     df_tmp = i_df.groupby('APPLICATION_NUMBER')[column].max()
#     df_tmp.columns = f'{column}_max'
#     i_df_main.join(df_tmp, on='APPLICATION_NUMBER', how='left')
    return(i_df_main)

In [256]:
# Количество активных кредитов 
bki.loc[bki['CREDIT_ACTIVE'] == 'Active'].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count()
# Количество закрытых кредитов 
bki.loc[bki['CREDIT_ACTIVE'] == 'Closed'].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count()

# Средний срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT'].mean()
# Минимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT'].min()
# Максимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT'].max()

# Средний срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_ENDDATE'].mean()
# Минимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_ENDDATE'].min()
# Максимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_ENDDATE'].max()

# Средний срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_ENDDATE_FACT'].mean()
# Минимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_ENDDATE_FACT'].min()
# Максимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['DAYS_ENDDATE_FACT'].max()

# Минимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['CREDIT_DAY_OVERDUE'].min()
# Максимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['CREDIT_DAY_OVERDUE'].max()
# Средняя просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['CREDIT_DAY_OVERDUE'].mean()

# Средний срок кредита
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_MAX_OVERDUE'].mean()
# Минимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_MAX_OVERDUE'].min()
# Максимальный срок кредита
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_MAX_OVERDUE'].max()

# Минимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM'].min()
# Максимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM'].max()
# Средняя просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM'].mean()

# Минимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_DEBT'].min()
# Максимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_DEBT'].max()
# Средняя просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_DEBT'].mean()

# Минимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_LIMIT'].min()
# Максимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_LIMIT'].max()
# Средняя просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_LIMIT'].mean()

# Минимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_OVERDUE'].min()
# Максимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_OVERDUE'].max()
# Средняя просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_OVERDUE'].mean()

# Минимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_UPDATE'].min()
# Максимальная просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_UPDATE'].max()
# Средняя просрочка по кредиту
bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_UPDATE'].mean()

APPLICATION_NUMBER
123423340    2
123423341    2
123423348    3
123423355    1
123423358    2
            ..
123779585    1
123779586    2
123779588    2
123779592    2
123779594    2
Name: APPLICATION_NUMBER, Length: 194407, dtype: int64

In [262]:
tmp = train.copy()

In [282]:
df_tmp = bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_UPDATE'].max()
df_tmp.columns = 'DAYS_CREDIT_UPDATE_max'
df_tmp
# tmp.join(df_tmp, on='APPLICATION_NUMBER', how='left')

APPLICATION_NUMBER
123423340      10.0
123423341    1185.0
123423342    2131.0
123423343     382.0
123423344     121.0
              ...  
123779588    2498.0
123779589     127.0
123779592     179.0
123779593     401.0
123779594    1621.0
Name: DAYS_CREDIT_UPDATE, Length: 273131, dtype: float64

In [307]:
bki.groupby('APPLICATION_NUMBER')['AMT_CREDIT_SUM_LIMIT'].mean()

APPLICATION_NUMBER
123423340       0.00000
123423341    7997.14125
123423342       0.00000
123423343       0.00000
123423344       0.00000
                ...    
123779588       0.00000
123779589       0.00000
123779592       0.00000
123779593           NaN
123779594       0.00000
Name: AMT_CREDIT_SUM_LIMIT, Length: 273131, dtype: float64

In [303]:
add_min_max_mean_param(tmp, bki, 'AMT_CREDIT_SUM_LIMIT')

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,{column}_mean,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_LIMIT_min,AMT_CREDIT_SUM_LIMIT_max
0,123687442,0,1,1.0,1.0,157500.0,855000.0,25128.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN
1,123597908,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123526683,0,1,3.0,0.0,135000.0,1006920.0,42660.0,3.0,1.0,...,0.0,0.0,0.0,7.0,0.0,4.0,NaN,NaN,NaN,NaN
3,123710391,1,1,1.0,0.0,180000.0,518562.0,22972.5,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,123590329,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110089,123672463,0,1,3.0,0.0,175500.0,269550.0,12618.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN
110090,123723001,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110091,123554358,0,1,3.0,2.0,270000.0,1024740.0,49428.0,5.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN


In [269]:
tmp

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,0,1,1.0,1.0,157500.0,855000.0,25128.0,1.0,1.0,...,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0
1,123597908,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123526683,0,1,3.0,0.0,135000.0,1006920.0,42660.0,3.0,1.0,...,2.0,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0
3,123710391,1,1,1.0,0.0,180000.0,518562.0,22972.5,1.0,1.0,...,2.0,NaN,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0
4,123590329,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110089,123672463,0,1,3.0,0.0,175500.0,269550.0,12618.0,1.0,1.0,...,2.0,0.528096,0.471774,0.581484,0.0,0.0,0.0,0.0,0.0,2.0
110090,123723001,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110091,123554358,0,1,3.0,2.0,270000.0,1024740.0,49428.0,5.0,1.0,...,4.0,0.653826,0.651227,0.558507,0.0,0.0,0.0,1.0,0.0,1.0


In [263]:
# Средняя просрочка по кредиту
tmp.join(bki.groupby('APPLICATION_NUMBER')['DAYS_CREDIT_UPDATE'].mean(), on='APPLICATION_NUMBER', how='left')

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT_UPDATE
0,123687442,0,1,1.0,1.0,157500.0,855000.0,25128.0,1.0,1.0,...,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,2162.000000
1,123597908,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249.777778
2,123526683,0,1,3.0,0.0,135000.0,1006920.0,42660.0,3.0,1.0,...,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,736.222222
3,123710391,1,1,1.0,0.0,180000.0,518562.0,22972.5,1.0,1.0,...,NaN,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0,1400.000000
4,123590329,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.000000
110089,123672463,0,1,3.0,0.0,175500.0,269550.0,12618.0,1.0,1.0,...,0.528096,0.471774,0.581484,0.0,0.0,0.0,0.0,0.0,2.0,167.000000
110090,123723001,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,959.333333
110091,123554358,0,1,3.0,2.0,270000.0,1024740.0,49428.0,5.0,1.0,...,0.653826,0.651227,0.558507,0.0,0.0,0.0,1.0,0.0,1.0,787.333333


In [ ]:
DAYS_CREDIT

In [260]:
bki.groupby('CREDIT_TYPE')['CREDIT_TYPE'].count()

CREDIT_TYPE
Another type of loan                               586
Car loan                                         15348
Cash loan (non-earmarked)                           31
Consumer credit                                 688431
Credit card                                     222238
Loan for business development                     1055
Loan for purchase of shares (margin lending)         4
Loan for the purchase of equipment                  14
Loan for working capital replenishment             261
Microloan                                         6788
Mobile operator loan                                 1
Mortgage                                         10161
Real estate loan                                    16
Unknown type of loan                               300
Name: CREDIT_TYPE, dtype: int64

In [172]:
# Объединяем train и bki
# train = 
train.join(bki.set_index('APPLICATION_NUMBER'), on='APPLICATION_NUMBER', how='left').shape
# train.head()
# Простое объединение не подходит, есть дубликаты по APPLICATION_NUMBER

(317272, 42)

In [81]:
app_hist.head()

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,...,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,...,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,...,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0
2,49791680,123445379,Cash,15060.735,112500.0,136444.5,NaN,112500.0,Approved,301,...,x-sell,-1,12.0,high,365243.0,271.0,59.0,365243.0,365243.0,1.0
3,50087457,123499497,Cash,47041.335,450000.0,470790.0,NaN,450000.0,Approved,512,...,x-sell,-1,12.0,middle,365243.0,482.0,152.0,182.0,177.0,1.0
4,49052479,123525393,Cash,31924.395,337500.0,404055.0,NaN,337500.0,Refused,781,...,walk-in,-1,24.0,high,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
app_hist.shape

(1670214, 26)

In [210]:
join_index = ['APPLICATION_NUMBER', 'PREV_APPLICATION_NUMBER']
app_hist.loc[app_hist.duplicated(join_index)]

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,...,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL


In [212]:
payments.shape

(1023932, 8)

In [222]:
unic_app = payments.loc[payments.duplicated(join_index), 'APPLICATION_NUMBER'].unique()

In [223]:
payments.loc[payments['APPLICATION_NUMBER'].isin(unic_app)].sort_values(join_index)

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
19819,48637907,123423340,2.0,4,1619.0,1628.0,17397.900,17397.900
355515,48637907,123423340,1.0,3,1649.0,1660.0,3951.000,3951.000
164049,48307032,123423341,1.0,8,355.0,375.0,9251.775,9251.775
397014,48307032,123423341,1.0,17,85.0,99.0,9251.775,9251.775
132910,48960247,123423346,1.0,7,164.0,166.0,16037.640,16037.640
...,...,...,...,...,...,...,...,...
941441,48447904,123779594,1.0,14,272.0,268.0,28873.170,27184.050
153451,49899598,123779594,3.0,13,396.0,404.0,27489.690,27489.690
260396,49899598,123779594,3.0,11,456.0,465.0,27489.690,27489.690
460610,49899598,123779594,3.0,12,426.0,435.0,27489.690,27489.690


In [106]:
count_app = app_hist.groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count()
count_app

APPLICATION_NUMBER
123423340    1
123423341    1
123423342    3
123423343    1
123423344    2
            ..
123779590    1
123779591    1
123779592    2
123779593    2
123779594    8
Name: APPLICATION_NUMBER, Length: 338857, dtype: int64

In [107]:
count_app = app_hist.groupby('PREV_APPLICATION_NUMBER')['APPLICATION_NUMBER'].count()
count_app

PREV_APPLICATION_NUMBER
48268215    1
48268216    1
48268217    1
48268218    1
48268219    1
           ..
50113591    1
50113592    1
50113593    1
50113595    1
50113596    1
Name: APPLICATION_NUMBER, Length: 1670214, dtype: int64

У нас есть данные о предыдущих заявках. Следовательно:

    1. Если у нас нет данных о клиенте, мы можем посмотреть их по предыдущим заявкам;
    
    2. Можем получить дополнительные данные: отношение одобренных заявкок к не одобренным, общее число заявок, средняя сумма заявок и т.д.;

Наборы данных payments и app_hist содержат поля по которым их можно объединить (APPLICATION_NUMBER, PREV_APPLICATION_NUMBER)

In [135]:
join_index = ['APPLICATION_NUMBER', 'PREV_APPLICATION_NUMBER']
app_hist = payments.join(app_hist.set_index(join_index), on=join_index, how='outer')

На основе этих данных можно расширить наш набор данных для обучения. Таргет выставлять будем следующим образом. Если есть информация о платеже, значит заявку одобрили, в противном случае заявку отклонили.

In [213]:
unique_app = app_hist['APPLICATION_NUMBER'].unique()

In [214]:
person = [[i, k] for i, k in enumerate(unique_app)]
person

[[0, 123595216],
 [1, 123431468],
 [2, 123445379],
 [3, 123499497],
 [4, 123525393],
 [5, 123522722],
 [6, 123499043],
 [7, 123619638],
 [8, 123665631],
 [9, 123657688],
 [10, 123771051],
 [11, 123484479],
 [12, 123581967],
 [13, 123645015],
 [14, 123593997],
 [15, 123474951],
 [16, 123477941],
 [17, 123466087],
 [18, 123719644],
 [19, 123522517],
 [20, 123489829],
 [21, 123591121],
 [22, 123625551],
 [23, 123522692],
 [24, 123599046],
 [25, 123662064],
 [26, 123756808],
 [27, 123741722],
 [28, 123639003],
 [29, 123493628],
 [30, 123505789],
 [31, 123581185],
 [32, 123716850],
 [33, 123578670],
 [34, 123531339],
 [35, 123501714],
 [36, 123484930],
 [37, 123448496],
 [38, 123604928],
 [39, 123581415],
 [40, 123570911],
 [41, 123511753],
 [42, 123627794],
 [43, 123442313],
 [44, 123604809],
 [45, 123770760],
 [46, 123559076],
 [47, 123579856],
 [48, 123524060],
 [49, 123622730],
 [50, 123486999],
 [51, 123576730],
 [52, 123694779],
 [53, 123520682],
 [54, 123706984],
 [55, 123614315],
 [

In [215]:
# вычисляем одинаковые предыдущие заявки для определения клиента
dublicate_app = app_hist.loc[app_hist['PREV_APPLICATION_NUMBER'].duplicated(), 'APPLICATION_NUMBER'].values

In [217]:
dublicate_app

array([], dtype=int64)

In [216]:
app_hist.loc[app_hist['APPLICATION_NUMBER'].isin(dublicate_app), ['APPLICATION_NUMBER', 'PREV_APPLICATION_NUMBER']]

,APPLICATION_NUMBER,PREV_APPLICATION_NUMBER


In [148]:
clien_profile.describe()

,APPLICATION_NUMBER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,2.500000e+05,250000.000000,2.500000e+05,2.500000e+05,249989.000000,250000.000000,250000.000000,250000.000000,85041.000000,250000.000000,...,249998.000000,109121.000000,2.494680e+05,200462.000000,216293.000000,216293.000000,216293.000000,216293.000000,216293.000000,216293.000000
mean,1.235771e+08,0.417624,1.690662e+05,5.993051e+05,27125.209585,0.020871,16033.460496,67505.883296,12.030221,0.280244,...,2.154025,0.502122,5.141749e-01,0.510848,0.006348,0.006866,0.034217,0.267475,0.266786,1.899030
std,8.877722e+04,0.722529,2.588480e+05,4.022720e+05,14503.930366,0.013827,4361.332320,139261.099609,11.899404,0.449119,...,0.911543,0.210817,1.910938e-01,0.195054,0.083783,0.107957,0.204029,0.914514,0.831547,1.868811
min,1.234233e+08,0.000000,2.565000e+04,4.500000e+04,1615.500000,0.000290,7489.000000,0.000000,0.000000,0.000000,...,1.000000,0.014691,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235002e+08,0.000000,1.125000e+05,2.700000e+05,16524.000000,0.010006,12415.000000,931.000000,5.000000,0.000000,...,2.000000,0.334180,3.920026e-01,0.370650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.235772e+08,0.000000,1.485000e+05,5.147775e+05,24930.000000,0.018850,15744.000000,2217.000000,9.000000,0.000000,...,2.000000,0.506018,5.656713e-01,0.535276,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.236540e+08,1.000000,2.025000e+05,8.086500e+05,34618.500000,0.028663,19681.000000,5678.000000,15.000000,1.000000,...,3.000000,0.674793,6.635195e-01,0.669057,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.237309e+08,19.000000,1.170000e+08,4.050000e+06,230161.500000,0.072508,25229.000000,365243.000000,69.000000,1.000000,...,20.000000,0.962693,8.549997e-01,0.896010,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


## Base line

In [62]:
all_zero = np.array(train.shape[0] * [1])
roc_auc_score(train['TARGET'], all_zero)

0.5

In [68]:
all_zero = np.random.randint(2, size=train.shape[0])
roc_auc_score(train['TARGET'], all_zero)

0.5014212712374523

## Split data set

In [233]:
object_features = train.select_dtypes(include=[np.object])
print(f"count of object_features {object_features.shape[1]}")

object_features_col = list(object_features.columns)
object_features_col

count of object_features 4


['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS']

In [237]:
# Обработка категориальных признаков
for feature in object_features:
    namber_val = 0
    for unic_val in train[feature].unique():
        namber_val += 1
        train.loc[train[feature] == unic_val, feature] = namber_val
#         leader_board.loc[leader_board[feature] == unic_val, feature] = namber_val
    train[feature] = pd.to_numeric(train[feature])
#     leader_board[feature] = pd.to_numeric(leader_board[feature])

In [238]:
target = 'TARGET'
df_train, df_test_val = train_test_split(
    train.drop(["APPLICATION_NUMBER"], axis=1), train_size=0.75, shuffle=True, random_state=1,
)

df_val, df_test = train_test_split(
    df_test_val, train_size=0.5, shuffle=True, random_state=1,
)

## XGBoost

In [241]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 100,
    "reg_lambda": 100,
    "max_depth": 10,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

In [239]:
dtrain = xgb.DMatrix(
    data=df_train.drop(target, axis=1), label=df_train[target]
)
dvalid = xgb.DMatrix(
    data=df_val.drop(target, axis=1), label=df_val[target]
)
dtest = xgb.DMatrix(
    data=df_test.drop(target, axis=1), label=df_test[target]
)

In [247]:
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=200,
    early_stopping_rounds=50,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=10,
    maximize=True,
)

[18:36:11] WARNING: /workspace/src/learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.58670	valid-auc:0.57498
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[10]	train-auc:0.68781	valid-auc:0.67306
[20]	train-auc:0.69979	valid-auc:0.68386
[30]	train-auc:0.71556	valid-auc:0.69847
[40]	train-auc:0.72355	valid-auc:0.70652
[50]	train-auc:0.72876	valid-auc:0.71028
[60]	train-auc:0.73148	valid-auc:0.71256
[70]	train-auc:0.73272	valid-auc:0.71335
[80]	train-auc:0.73272	valid-auc:0.71335
[90]	train-auc:0.73272	valid-auc:0.71335
[100]	train-auc:0.73272	valid-auc:0.71335
[110]	train-auc:0.73272	valid-auc:0.71335
Stopping. Best iteration

In [248]:
predict_val = model.predict(dtest)
roc_auc_score(df_test[target], predict_val)

0.7089824849522534